In [6]:
from fileToolbox import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [7]:
def warn(*args, **kwargs):
    pass


import warnings

warnings.warn = warn
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
set_of_person_ids = set(train_df["person_id"])
list_of_unique_person_ids = list(set_of_person_ids) # osoby to będą moje x

In [9]:
set_of_movie_ids = set(train_df["movie_id"])
list_of_unique_movie_ids = list(set_of_person_ids) # osoby to będą moje x

In [10]:
tmp_p = set(task_df["person_id"])
tmp_m = set(task_df["movie_id"])

Sprawdzenie czy napewno wszystkie person_id i movie_id z taska występują w trainie

In [11]:
for person in set_of_person_ids:
    if person not in tmp_p:
        print(person)

In [12]:
for movie in set_of_movie_ids:
    if movie not in tmp_m:
        print(movie)

Dla każdego filmu generuje n elementowy wektor cech

In [13]:
N = 10 # to będzie hiperparametr

In [14]:
M = len(set_of_movie_ids)
R = len(set_of_person_ids)

In [15]:
print(M)
print(R)

200
358


In [91]:
import random
# losowa inicjalizacja list p - parametrów i x - wejść z przedziału <-1, 1>
X_INPUTS = [[random.uniform(-1, 1) for j in range(N+1)] for i in range(M)] # parametrów jest n+1 bo stała
PARAMS = [[random.uniform(-1, 1) for j in range(N)] for i in range(R)]

In [19]:
# kodowanie filmów czyli m idzie od 0 do M, ale m to nie jest movie_id wieć muszę zrobić ,np. m=0 -> movie_id = 12
movies_dict = {}
for m in range(M):
    movies_dict[m] = list(set_of_movie_ids)[m]

In [ ]:
movies_dict

In [56]:
with open('json/movies_dict.json', 'w') as file:
    json.dump(movies_dict, file)

In [21]:
# kodowanie filmów czyli m idzie od 0 do M, ale m to nie jest movie_id wieć muszę zrobić ,np. m=0 -> movie_id = 12
person_dict = {}
for r in range(R):
    person_dict[r] = list(set_of_person_ids)[r]

In [ ]:
person_dict

In [55]:
with open('json/person_dict.json', 'w') as file:
    json.dump(person_dict, file)

In [50]:
def findMovieRatingByPerson(person_id, movie_id):
    person_movies = filterDataFrame("person_id", person_id, train_df)
    if movie_id in list(person_movies["movie_id"]):
        return int(person_movies[person_movies["movie_id"]==movie_id]["rate"])
    else:
        return None

In [59]:
# macierz Y - ustawić rate lub NULL na poszczególnych indeksach
Y_RESULTS = []
E = [] # tablica do przechowywania maski 0 i 1, czy m-ty film jest oceniony przez m-tą osobę
for m in range(M):
    movie_id = movies_dict[m]
    movie_ratings = []
    E_row = []
    for r in range(R):
        # znalezienie oceny dla m-tego filmu, wystawionej przez r-tą osobę
        person_id = person_dict[r]
        rate = findMovieRatingByPerson(person_id, movie_id)
        movie_ratings.append(rate)
        if rate is None:
            E_row.append(0)
        else:
            E_row.append(1)
    Y_RESULTS.append(movie_ratings)
    E.append(E_row)


In [54]:
with open('json/Y_RESULTS.json', 'w') as file:
    json.dump(Y_RESULTS, file)

In [61]:
with open('json/E.json', 'w') as file:
    json.dump(E, file)

In [29]:
def isMovieRatedByPerson(movie, person):
    df_person = filterDataFrame("person_id", person, train_df)
    list_of_movies = list(df_person["movie_id"]))
    if movie in list_of_movies:
        return 1
    else:
        return 0

In [2]:
# def main_function(p, x, y, alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
#     for epoch in range(num_of_epochs):
#         for r, p in enumerate(PARAMS):
#             for m in range(len(movies)): # m - liczba filmów, p - parametry da r-tej ososby
#                 for n in range(len(p)):
#                     p[n] = p[n] - alfa * derivative_P(x[m], y, p, n) # poprawiam naraz w jednej iteracji x i p
#                     x[m][n] = x[n] - beta * derivative_X(x[m], y, p, n)

In [76]:
def main_function(alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
    for epoch in range(num_of_epochs):
        for r, p in enumerate(PARAMS): # r - indeks osób
            for m, x in enumerate(X_INPUTS): # m - indeks filmów
                if E[m][r]==1: # używamy tylko tych danych, gdzie znany jest rate dla m-tego filmu i r-tej osoby
                    for n in range(len(p)):
                        # poprawiam naraz w jednej iteracji x i p
                        p[n] = p[n] - alfa * derivative_P(X_INPUTS, Y_RESULTS, p, n, r) # UWAGA parametrów jest n+1
                        if n != (len(p)-1): # p jest o 1 dłuższe niż x
                            x[n] = x[n] - beta * derivative_X(x, Y_RESULTS, PARAMS, n, m)
    print(Q(X_INPUTS, PARAMS, Y_RESULTS)) # błąd liczymy po całej przestrzeni

In [77]:
def Q(X, P, Y):
    res = 0.0
    for r, p in enumerate(P): # r - indeks osób
            for m, x in enumerate(X):
                res += (calc_output(x, p) - Y[r][m]) ** 2
    return res / 2 

In [89]:
def derivative_P(X, Y, p, i, r):
    res = 0.0
    for m in range(len(X)): # przechodzimy po filmach m
        if E[m][r]==1:
            if i ==0: # dla p0 bez mnożenia z X[m][i]
                res += (calc_output(X[m], p) - Y[m][r])
            else:
                tmp = calc_output(X[m], p)
                tmp2 = X[m][i-1]
                res += (tmp - Y[m][r]) * tmp2
    return res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [85]:
def derivative_X(X, Y, p, i, m):
    res = 0.0
    for r in range(len(p)): # przechodzimy po osobach r
        if E[m][r]==1:
            res += (calc_output(X, p[r]) - Y[m][r]) * p[r][i+1]
    return res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [93]:
def calc_output(x, p):
    summ = p[0]
    for n in range(len(x)):
        tmp = x[n]
        summ = summ + tmp * p[n+1] # muszę mnożyć x0 z p1, x1 z p2 itp. bo p o 1 dłuższe niż x
    return summ

In [95]:
main_function(0.5, 0.5, 10)

IndexError: list index out of range